### Burnt Area computation 

This notebook takes two Sentinel-2 products and an area of interest and will generate a GeoTIFF and RGBA as output.

In [ ]:
import cioppy
import snappy
import json

import sys
import os
from py_snap_helpers import op_help, get_operator_default_parameters, GraphProcessor

from geopandas import GeoDataFrame
import pandas as pd


from shapely.geometry import box
from shapely.wkt import loads
from shapely.errors import WKTReadingError
import numpy as np
import datetime
import math

import lxml.etree as etree
import gdal
from os.path import exists
from base64 import b64encode
import shutil

from helpers import *

gdal.UseExceptions()

#### Make the correct settings (area of interest, identifiers, input references, data path and environment variables, including paths)

In [ ]:
aoi_wkt = {
    'id': 'aoi',
    'value': 'POLYGON((-100.523 25.314,-100.523 25.427,-100.225 25.427,-100.225 25.314,-100.523 25.314))',
    'title': 'Area of interest in WKT',
    'abstract': 'Area of interest using a polygon in Well-Known-Text format or bounding box'
}

input_identifiers = (
    'S2B_MSIL2A_20210305T171109_N0214_R112_T14RLP_20210305T212158',
    'S2B_MSIL2A_20210424T170839_N0300_R112_T14RLP_20210424T211236'
) 

input_references = (
    'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20210305T171109_N0214_R112_T14RLP_20210305T212158',
    'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20210424T170839_N0300_R112_T14RLP_20210424T211236'
)

data_path = '/workspace/data'

os.environ['PREFIX'] = '/home/centos/.conda/envs/env_burned_area'
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
gpt_path = os.environ['GPT_BIN']
os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

sys.path.append('/home/centos/.conda/envs/env_burned_area/snap/.snap/snap-python')

In [ ]:
ciop = cioppy.Cioppy()

temp_results = []

sentinel2_endpoint = 'https://catalog.terradue.com/sentinel2/description'

for index, self in enumerate(input_references):
    
    search_params = dict()

    search_params['uid'] = input_identifiers[index]

    temp_results.append(
        ciop.search(
            end_point=sentinel2_endpoint,
            params=search_params,
            output_fields='identifier,self,wkt,startdate,enddate,enclosure,orbitDirection,cc', 
            model='EOP'
        )[0]
    )
    

for r in temp_results:
    r['wkt'] = loads(r['wkt'])

sentinel2_search = GeoDataFrame(temp_results)


aoi = None

if aoi_wkt['value'] != '':

    try:
        aoi = loads(aoi_wkt['value'])

        min_lon, min_lat, max_lon, max_lat =  list(aoi.bounds)
    
    except WKTReadingError:
        
        min_lon, min_lat, max_lon, max_lat = [float(c) for c in aoi_wkt['value'].split(',')]
        aoi = box(min_lon, min_lat, max_lon, max_lat)

for r in temp_results:
    analyse(r, aoi, data_path)

In [ ]:
sentinel2_search

In [ ]:
pre_processed = []

resample = dict()
resample['referenceBandName'] = 'B2'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

subset = dict()
subset['geoRegion'] = box(*aoi.bounds).wkt
subset['copyMetadata'] = 'true'

bands = '''<targetBands>
    <targetBand>
      <name>NDWI</name>
      <type>float32</type>
      <expression>(B3 - B8) / (B3 + B8)</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>NBR</name>
      <type>float32</type>
      <expression>(B8 - B12) / (B8 + B12)</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>valid_pixels</name>
      <type>float32</type>
      <expression>scl_vegetation or scl_not_vegetated ? 1 : 0</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    </targetBands>'''

band_maths = dict()
band_maths['targetBandDescriptors'] = bands 


for row in temp_results:
    row['pre_proc'] = ''

#for index, row in sentinel2_search.iterrows():
for row in temp_results:
    
    print(os.path.join(row['local_path'], row['identifier'] + '.SAFE', 'MTD_MSIL2A.xml'))
    
    read = dict()
    read['file'] = os.path.join(row['local_path'], row['identifier'] + '.SAFE', 'MTD_MSIL2A.xml') 
    write = dict()
    write['file'] = 'pre_{}'.format(row['identifier'])


    row['pre_proc'] = 'pre_{}'.format(row['identifier'])
    print("PRE_PROC: {0}".format(row['pre_proc']))
    
    pre_processed.append('pre_{}'.format(row['identifier']))
    
    pre_processing(Read=read, 
        Resample=resample, 
        Reproject=reproject, 
        Subset=subset,
        BandMaths=band_maths,
        Write=write
    )

In [ ]:
dims = sorted(["{0}.dim".format(row['pre_proc']) for row in temp_results], key=lambda s: s[15:30])

master_path = dims[0]
slave_path = dims[1]

print("Master: {0}".format(master_path))
print("Slave:  {0}".format(slave_path))

mygraph = GraphProcessor(gpt_path)
operator = 'Read'

node_id = 'Read_M'

source_node_id = ''

parameters = get_operator_default_parameters(operator)
     
parameters['file'] = master_path 
    
mygraph.add_node(
    node_id,
    operator,
    parameters, source_node_id
)


operator = 'Read'

node_id = 'Read_S'

source_node_id = ''

parameters = get_operator_default_parameters(operator)
     
parameters['file'] = slave_path   
    
mygraph.add_node(
    node_id, 
    operator,
    parameters, source_node_id
)

operator = 'Collocate'

parameters = get_operator_default_parameters(operator)

parameters['masterComponentPattern'] = 'PRE_FIRE_${ORIGINAL_NAME}'
parameters['slaveComponentPattern'] = 'POST_FIRE_${ORIGINAL_NAME}'
parameters['targetProductName']='pre_post_collocated'
source_node_id = dict()

source_node_id['master'] = 'Read_M'

source_node_id['slave'] = 'Read_S'


node_id = 'Collocate'

mygraph.add_node(
    node_id,
    operator, 
    parameters, ['Read_M','Read_S']
)

operator = 'Write'

node_id = 'Write'

source_node_id = 'Collocate'


parameters = get_operator_default_parameters(operator)

parameters['file'] = 'collocated'
parameters['formatName'] = 'BEAM-DIMAP'

mygraph.add_node(node_id, operator, parameters, source_node_id)

print("Run graph (start)")
#mygraph.save_graph('graph_collocate.xml')
mygraph.run()

print("Run graph (end)")


In [ ]:
output_name = 'burned_area_{0}_{1}'.format(dims[0][15:23], dims[1][15:23])

In [ ]:
output_name

In [ ]:
collocated_input = 'collocated.dim'

read = dict()
read['file'] = collocated_input

bands = '''<targetBands>
    <targetBand>
      <name>dNBR</name>
      <type>float32</type>
      <expression>(PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1 and ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27) ? PRE_FIRE_NBR - POST_FIRE_NBR : -999</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>RBR</name>
      <type>float32</type>
      <expression>(PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1 and ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27) ? ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) : -999</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>valid_pixels</name>
      <type>float32</type>
      <expression>PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    </targetBands>'''

band_maths = dict()
band_maths['targetBandDescriptors'] = bands 

write = dict()
write['file'] = output_name
write['formatName'] = 'GeoTIFF'

burned_area(
    Read=read, 
    BandMaths=band_maths,
    Write=write
)

In [ ]:
create_rgba(
    output_name + '.tif',
    output_name + '.rgb.tif'
)

In [ ]:
band_names = [
    'dNBR',
    'RBR',
    'valid_pixels'
]

expressions = [
    'PRE_FIRE_NDWI >= 0.0 ? 0 : ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27 ? PRE_FIRE_NBR - POST_FIRE_NBR : 0',
    'PRE_FIRE_NDWI >= 0.0 ? 0 : ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27 ? ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) : 0',
    'PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1'
]

metadata= dict()

ds_temp = gdal.Open(output_name + '.tif',  gdal.OF_UPDATE)

for band_index in range(ds_temp.RasterCount):

    metadata= dict()
    metadata['BAND_EXPRESSION'] = expressions[band_index]

    src_band = ds_temp.GetRasterBand(band_index+1)
    src_band.SetMetadata(metadata)
    src_band.SetDescription(band_names[band_index])  

ds_temp.FlushCache()

END